In [1]:
import pandas as pd
import re

In [2]:
with open("../files/input/clusters_report.txt", "r", encoding="utf-8") as f:
    lines = f.readlines()

In [3]:
print(lines)
# lines

['Cluster  Cantidad de     Porcentaje de   Principales palabras clave     \n', '         palabras clave  palabras clave\n', '         \n', '---------------------------------------------------------------------------------------------------------------------\n', '   1     105             15,9 %          maximum power point tracking, fuzzy-logic based control, photo voltaic (pv), \n', '                                         photo-voltaic  system,  differential   evolution   algorithm,   evolutionary\n', '                                         algorithm, double-fed induction generator (dfig), ant  colony  optimisation, \n', '                                         photo voltaic array, firefly algorithm, partial shade.\n', '\n', '   2     102             15,4 %          support vector machine,  long  short-term  memory,  back-propagation  neural\n', '                                         network,  convolution  neural  network,  speed   wind   prediction,   energy\n', '             

In [4]:
columns = ["cluster", "cantidad_de_palabras_clave", "porcentaje_de_palabras_clave", "principales_palabras_clave"]
columns

['cluster',
 'cantidad_de_palabras_clave',
 'porcentaje_de_palabras_clave',
 'principales_palabras_clave']

In [5]:
import re

clusters_raw = []  # lista donde guardaremos cada cluster completo
current_cluster = ""  # texto acumulado

for line in lines[4:]:
    # Si la línea comienza con un número → nuevo cluster
    if re.match(r'^\s*\d+', line):
        # Guardar el cluster anterior si ya teníamos uno
        if current_cluster != "":
            clusters_raw.append(current_cluster.strip())
        # Empezar nuevo cluster
        current_cluster = line.strip()
    else:
        # Continuación del texto anterior → agregarlo
        current_cluster += " " + line.strip()

# Agregar el último cluster
if current_cluster:
    clusters_raw.append(current_cluster.strip())


In [6]:
# clusters_raw

In [7]:
filas = [fila.split('\t') for fila in clusters_raw]
# filas

In [8]:
import re

# Lista donde guardaremos las filas ya separadas
filas_limpias = []

# Expresión regular, cada () representa una columna
# \S+ : uno o más caracteres que no son espacios
# \s+ : uno o más espacios

patron = r'^(\S+)\s+(\S+)\s+(\S+\s*%)\s+(.+)$'

for fila in clusters_raw:
    match = re.match(patron, fila)
    if match:
        # Extrae los 4 grupos (columnas)
        columnas_match = list(match.groups())
        filas_limpias.append(columnas_match)
    
    else:
        print("No fue posible match para la fila:", fila)

# filas_limpias

In [9]:
df = pd.DataFrame(filas_limpias, columns=columns)
# df

In [10]:
df.porcentaje_de_palabras_clave = df.porcentaje_de_palabras_clave.str.replace('%', '')
df.porcentaje_de_palabras_clave = df.porcentaje_de_palabras_clave.str.replace(',', '.')
df.porcentaje_de_palabras_clave = df.porcentaje_de_palabras_clave.astype(float)
df.cantidad_de_palabras_clave = df.cantidad_de_palabras_clave.astype(int)
# df

In [11]:
df.cantidad_de_palabras_clave.dtype

dtype('int64')

In [ ]:
df.principales_palabras_clave = (
    df.principales_palabras_clave
    .apply(lambda x: re.sub(r"\s+", " ", x.strip()))     # normaliza espacios
    .apply(lambda x: re.sub(r"\s*,\s*", ", ", x))        # coma + espacio
)
df.principales_palabras_clave = (df.principales_palabras_clave.str.replace(r"\s+", ' ')
                                   .str.replace(', ', ',')
                                   .str.replace(',', ', ')
                                   .str.strip()
                                   .str.replace('.', '')
)                              

In [13]:
# df.principales_palabras_clave = df.principales_palabras_clave.str.replace('.', '')
# df.principales_palabras_clave = df.principales_palabras_clave.str.replace('  ', ' ')
# df.principales_palabras_clave = df.principales_palabras_clave.str.replace('   ', ' ')
# df.principales_palabras_clave = df.principales_palabras_clave.str.replace('    ', ' ')
# df.principales_palabras_clave = df.principales_palabras_clave.str.replace('     ', ' ')
# df.principales_palabras_clave = df.principales_palabras_clave.str.replace('      ', ' ')
# df.principales_palabras_clave = df.principales_palabras_clave.str.replace('       ', ' ')
# df.principales_palabras_clave = df.principales_palabras_clave.str.strip("'\"")
# df

In [17]:
df.principales_palabras_clave[0]

'maximum power point tracking, fuzzy-logic based control, photo voltaic (pv), photo-voltaic system, differential evolution algorithm, evolutionary algorithm, double-fed induction generator (dfig), ant colony optimisation, photo voltaic array, firefly algorithm, partial shade'

In [18]:
# print(type(df.principales_palabras_clave[0]))


df.principales_palabras_clave.to_list()[0]

'maximum power point tracking, fuzzy-logic based control, photo voltaic (pv), photo-voltaic system, differential evolution algorithm, evolutionary algorithm, double-fed induction generator (dfig), ant colony optimisation, photo voltaic array, firefly algorithm, partial shade'